In [6]:
#download data from open data toronto
install.packages("opendatatoronto")
library(opendatatoronto)
library(dplyr)
 
# get package
package <- show_package("traffic-volumes-at-intersections-for-all-modes")
package
 
# get all resources for this package
resources <- list_package_resources("traffic-volumes-at-intersections-for-all-modes")
 
# identify datastore resources; by default, Toronto Open Data sets datastore resource format to CSV for non-geospatial and GeoJSON for geospatial resources
datastore_resources <- filter(resources, tolower(format) %in% c('csv', 'geojson'))
 
# load the first datastore resource as a sample
data <- filter(datastore_resources, row_number()==1) %>% get_resource()
data

Installing package into 'C:/Users/heche/AppData/Local/R/win-library/4.3'
(as 'lib' is unspecified)

also installing the dependencies 'triebeard', 'proxy', 'urltools', 'httpcode', 'mime', 'tidyselect', 'blob', 'tidyr', 'rematch', 'hms', 'prettyunits', 'e1071', 'wk', 'DBI', 'crul', 'dplyr', 'dbplyr', 'cellranger', 'progress', 'classInt', 'Rcpp', 's2', 'units', 'pkgconfig', 'ckanr', 'readxl', 'sf', 'tibble', 'curl'


Warning message in download.file(url, destfile, method, mode = "wb", ...):
"downloaded length 0 != reported length 0"
Warning message in download.file(url, destfile, method, mode = "wb", ...):
"URL 'https://cran.r-project.org/bin/windows/contrib/4.3/sf_1.0-15.zip': Timeout of 60 seconds was reached"


Error in download.file(url, destfile, method, mode = "wb", ...) : 
  download from 'https://cran.r-project.org/bin/windows/contrib/4.3/sf_1.0-15.zip' failed


Warning message in download.packages(pkgs, destdir = tmpd, available = available, :
"download of package 'sf' failed"


package 'triebeard' successfully unpacked and MD5 sums checked
package 'proxy' successfully unpacked and MD5 sums checked
package 'urltools' successfully unpacked and MD5 sums checked
package 'httpcode' successfully unpacked and MD5 sums checked
package 'mime' successfully unpacked and MD5 sums checked
package 'tidyselect' successfully unpacked and MD5 sums checked
package 'blob' successfully unpacked and MD5 sums checked
package 'tidyr' successfully unpacked and MD5 sums checked
package 'rematch' successfully unpacked and MD5 sums checked
package 'hms' successfully unpacked and MD5 sums checked
package 'prettyunits' successfully unpacked and MD5 sums checked
package 'e1071' successfully unpacked and MD5 sums checked
package 'wk' successfully unpacked and MD5 sums checked
package 'DBI' successfully unpacked and MD5 sums checked
package 'crul' successfully unpacked and MD5 sums checked
package 'dplyr' successfully unpacked and MD5 sums checked
package 'dbplyr' successfully unpacked and 

Warning message:
"package 'opendatatoronto' was built under R version 4.3.3"
Warning message:
"package 'dplyr' was built under R version 4.3.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




title,id,topics,civic_issues,publisher,excerpt,dataset_category,num_resources,formats,refresh_rate,last_refreshed
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<date>
Traffic Volumes at Intersections for All Modes,traffic-volumes-at-intersections-for-all-modes,Transportation,Mobility,Transportation Services,"This dataset includes all available Turning Movement Count traffic volume data observed at locations across the City. The data is separated by location, date of count, mode of transportation, direction, and movement pattern (if available).",Table,28,"JSON,CSV,XML",Daily,2024-03-07


_id,location_id,location,lng,lat,centreline_type,centreline_id,px,latest_count_date
<int>,<int>,<chr>,<dbl>,<dbl>,<int>,<int>,<int>,<chr>
1,1146,ELLESMERE RD AT PARKINGTON CRES,-79.24625,43.77332,2,13446642,2296,2022-12-17
2,1981,YORK MILLS RD E/B & W/B TO DON VALLEY PKWY N/B,-79.33466,43.75734,1,440171,NA,2004-04-08
3,3468,PARKSIDE DR N/B S OF SPRING RD,-79.45444,43.64051,1,30010748,NA,2021-12-15
4,3925,RIPLEY AVE AT SOUTH KINGSWAY,-79.47527,43.63678,2,13468657,NA,2022-01-27
5,3926,BLOOR ST AT SOUTH KINGSWAY & RIVERVIEW GARDENS (PX 334),-79.48575,43.64831,2,13467247,334,2022-05-10
6,3927,LAKE SHORE BLVD AT LESLIE ST (PX 206),-79.32844,43.65924,2,13465121,206,2013-05-27
7,3928,BRITISH COLUMBIA RD AT LAKE SHORE BLVD (PX 1344),-79.42918,43.63143,2,13469154,1344,2018-01-24
8,3929,JAMESON AVE AT LAKE SHORE BLVD W/B (PX 224),-79.43456,43.63334,2,13468970,224,2017-10-03
9,3930,LAKE SHORE BLVD AT ONTARIO DR & REMEMBRANCE DR (PX 930),-79.41978,43.63043,2,13469216,930,2018-01-24
